# OpenClassrooms - Ingenieur IA
# Projet 7 - AirParadis
# Détectez les Bad Buzz grâce au Deep Learning

## Objectif du projet : 
- **Développer le prototype d’un produit IA permettant de prédire le sentiment associé à un tweet**

## Trois approches :
- **Approche 1 : 'API sur étagère' en utilisant l’API du service cognitif proposé par Microsoft Azure pour l’analyse de sentiment**
- **Approche 2 : 'Modèle sur mesure simple' en utilisant le service Azure Machine Learning Studio**
- **Approche 3 : 'Modèle sur mesure avancé' en utilisant le service Azure Machine Learning pour développer un modèle basé sur des réseaux de neurones profonds pour prédire le sentiment associé à un tweet**

## Plan - Approche 3 : 'Modèle sur mesure avancé' :
- **Déploiement dans Azure**
    - Récupération du workspace Azure ML
    - Récupération du modèle de Azure ML
    - Création du fichier score.py
    - Configuration de l'environnement d'éxécution
    - Configuration de l'inférence
    - Déploiement en local
    - Déploiement en remote
- **Prototype permettant de prédire le sentiment associé à un tweet**
    - Fonction 'prototype'
    - Test du déploiement dans Azure

In [1]:
import io
import os
import json
import requests

import tensorflow as tf
from tensorflow.keras.models import load_model
from keras_preprocessing.text import tokenizer_from_json

from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Workspace
from azureml.core import Webservice
from azureml.core.webservice import LocalWebservice
import warnings
warnings.filterwarnings('ignore')

# Déploiement dans Azure 

## Récupération du workspace Azure ML

In [2]:
azure_subscription_id = os.environ['AZURE_SUBSCRIPTION_ID']

In [3]:
ws = Workspace(subscription_id=azure_subscription_id,
              resource_group="OC-IA-P7",
              workspace_name="ws-OC-P7")

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


## Récupération du modèle de Azure ML

In [4]:
model_name_azure = 'Best_Model_AirParadis_h5'
model_azure = Model(ws, model_name_azure)
model_path_azure = model_azure.download(exist_ok = True)

In [5]:
model_azure

Model(workspace=Workspace.create(name='ws-OC-P7', subscription_id='dc0050bb-8e50-4b60-8aac-034371ba1a2a', resource_group='OC-IA-P7'), name=Best_Model_AirParadis_h5, id=Best_Model_AirParadis_h5:1, version=1, tags={}, properties={})

In [6]:
model_path_azure

'airparadis_best_model.h5'

In [7]:
model_keras = load_model(model_path_azure)

In [8]:
model_keras.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 24, 100)           3606300   
_________________________________________________________________
dropout (Dropout)            (None, 24, 100)           0         
_________________________________________________________________
gru (GRU)                    (None, 50)                22800     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 3,629,151
Trainable params: 22,851
Non-trainable params: 3,606,300
_________________________________________________________________


##### Remarque :
- Le modèle a été enregistré dans les modèles Azure avec le code suivant :

In [9]:
"""
model = Model.register(model_path = "airparadis_best_model.h5",
                       model_name = "Best_Model_AirParadis_h5",
                       description = "Best model for AirParadis",
                       workspace = ws)
"""                       

'\nmodel = Model.register(model_path = "airparadis_best_model.h5",\n                       model_name = "Best_Model_AirParadis_h5",\n                       description = "Best model for AirParadis",\n                       workspace = ws)\n'

## Création du fichier score.py

In [10]:
%%writefile score.py
import numpy as np
import preprocessor as tweetpreprocessor

import tensorflow
from keras_preprocessing.text import tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

from azure.storage.blob import BlobServiceClient
from azureml.core.model import Model

import json
import os

def init():
    global model
    model_path = Model.get_model_path(model_name = 'Best_Model_AirParadis_h5')
    model = load_model(model_path)

def run(request):
    tweet_request = json.loads(request)["tweet"]
    X_test_pad = preprocess(tweet_request)
    y_pred = model.predict(X_test_pad)
    y_pred_classe = np.where(y_pred>0.5, "positive", "negative")

    return f"Sentiment is : {y_pred_classe} - score = {y_pred}"

def preprocess(tweet):
    tweet_clean = tweetpreprocessor.clean(tweet)
    
    connect_str = os.getenv('AZURE_STORAGE_CONNECTION_STRING')
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container="airparadiscontainer", blob='tokenizer.json')
    
    with open('tokenizer.json', "wb") as download_file:
        download_file.write(blob_client.download_blob().readall())
    
    with open('tokenizer.json') as f:
        data = json.load(f)
        tokenizer = tokenizer_from_json(data)
    
    MAX_WORD_LENGTH = 24
    trunc_type='post'
    padding_type='post'
    
    tweet_clean_sequence = tokenizer.texts_to_sequences([tweet_clean])
    tweet_clean_pad = pad_sequences(tweet_clean_sequence, maxlen=MAX_WORD_LENGTH, padding=padding_type, truncating=trunc_type)

    return tweet_clean_pad

Overwriting score.py


##### Remarque :
- Le tokenizer a été enregistré dans le service de stockage Azure Blob avec le code suivant :

In [11]:
"""
from azure.storage.blob import BlobServiceClient
    
# 1 - Récupération de la chaine de connection au service dans les variables d'environnement
connect_str = os.getenv('AZURE_STORAGE_CONNECTION_STRING')

# 2 - Création d'un client pour accéder au service de stockage Azure Blob
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

# 3 - Upload du token dans le service de stockage
blob_client = blob_service_client.get_blob_client(container="airparadiscontainer", blob='tokenizer.json')
with open('tokenizer.json', "rb") as data:
    blob_client.upload_blob(data)
"""    

'\nfrom azure.storage.blob import BlobServiceClient\n    \n# 1 - Récupération de la chaine de connection au service dans les variables d\'environnement\nconnect_str = os.getenv(\'AZURE_STORAGE_CONNECTION_STRING\')\n\n# 2 - Création d\'un client pour accéder au service de stockage Azure Blob\nblob_service_client = BlobServiceClient.from_connection_string(connect_str)\n\n# 3 - Upload du token dans le service de stockage\nblob_client = blob_service_client.get_blob_client(container="airparadiscontainer", blob=\'tokenizer.json\')\nwith open(\'tokenizer.json\', "rb") as data:\n    blob_client.upload_blob(data)\n'

## Configuration de l'environnement d'éxécution

In [12]:
env = Environment("myenv")
python_packages = ['numpy', 'tensorflow','azureml-core', 'keras', 'tweet-preprocessor', 'azure-storage-blob']
for package in python_packages:
    env.python.conda_dependencies.add_pip_package(package)

In [13]:
connect_str = os.getenv('AZURE_STORAGE_CONNECTION_STRING')

In [14]:
env.environment_variables = {"AZURE_STORAGE_CONNECTION_STRING":connect_str}

## Configuration de l'inférence

In [15]:
inference_config = InferenceConfig(environment= env, entry_script= "score.py")

## Déploiement en local

##### Remarque :
- Code à décommenter pour un déploiement en local

In [16]:
"""
deployment_config_local = LocalWebservice.deploy_configuration(port=6789)

service_local = Model.deploy(ws, 'airparadisdeploymentlocal', [model_azure], inference_config, deployment_config_local)

service_local.wait_for_deployment(show_output = True)
print(service_local.state)
"""

"\ndeployment_config_local = LocalWebservice.deploy_configuration(port=6789)\n\nservice_local = Model.deploy(ws, 'airparadisdeploymentlocal', [model_azure], inference_config, deployment_config_local)\n\nservice_local.wait_for_deployment(show_output = True)\nprint(service_local.state)\n"

## Déploiement en remote

In [17]:
deployment_config_remote = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, description = 'AirParadis Deployment')

service_remote = Model.deploy(ws, 'airparadisdeployment3', [model_azure], inference_config, deployment_config_remote)

service_remote.wait_for_deployment(show_output = True)
print(service_remote.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-08-24 15:30:13+02:00 Creating Container Registry if not exists.
2021-08-24 15:30:13+02:00 Registering the environment.
2021-08-24 15:30:15+02:00 Use the existing image.
2021-08-24 15:30:16+02:00 Generating deployment configuration.
2021-08-24 15:30:16+02:00 Submitting deployment to compute.
2021-08-24 15:30:21+02:00 Checking the status of deployment airparadisdeployment3..
2021-08-24 15:33:06+02:00 Checking the status of inference endpoint airparadisdeployment3.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


# Prototype permettant de prédire le sentiment associé à un tweet
- Ce prototype permet de prédire le sentiment associé à un tweet en utilisant le modèle sur mesure avancé déployé en remote dans Azure

## Fonction 'prototype' :
- prend en argument un Tweet
- renvoie la prédiction du sentiment et le score de probabilité associé au Tweet en entrée

In [18]:
def get_sentiment(tweet):
    # 1 - On récupère l'URI du service
    service = Webservice(workspace=ws, name="airparadisdeployment3")
    scoring_uri = service.scoring_uri

    # 2 - On récupère la clé d'authentification dans les variables d'environnement
    key = os.environ['KEY_AIRPARADIS']

    # 3 - On prépare le header
    headers = {"Content-Type": "application/json"}
    headers["Authorization"] = f"Bearer {key}"

    # 4 - On envoie la requête et on récupère la réponse
    data = {"tweet":tweet}
    data = json.dumps(data)
    resp = requests.post(scoring_uri, data=data, headers=headers)

    # 5 - On affiche le résultat = sentiment ('positive' ou 'negative'), ainsi que le score correspondant
    print(resp.text)

## Test du déploiement dans Azure

#### Tests avec les mêmes Tweets que le modèle avancé développé avec Jupyter

##### Tweet positif

In [19]:
get_sentiment("The service at AirParadis is good")

"Sentiment is : [['positive']] - score = [[0.9623542]]"


##### Tweet négatif

In [20]:
get_sentiment("The service at AirParadis is bad")

"Sentiment is : [['negative']] - score = [[0.03483599]]"


##### Tweet neutre

In [21]:
get_sentiment("The service at AirParadis is average")

"Sentiment is : [['negative']] - score = [[0.47108507]]"


#### Conclusion :
- Le déploiement fonctionne
- On retrouve bien les mêmes prédictions et les mêmes scores de probabilité qu'avec le modèle avancé du Jupyter Notebook